In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import precision_recall_curve, recall_score

In [223]:
churn_data = pd.read_csv('/Users/leahdeyoung/Desktop/GitHub/beta-bank-practicum/Churn.csv', encoding = "utf-8")


In [224]:
display(churn_data.sample(10))
churn_data.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9007,9008,15684934,Rose,726,France,Male,31,9.0,0.00,2,1,1,106117.30,0
933,934,15737795,Scott,512,Spain,Male,36,1.0,0.00,1,0,1,135482.26,1
6950,6951,15667392,L?,652,Spain,Female,38,NaN,123081.84,2,1,1,188657.97,0
1155,1156,15768006,Wu,729,France,Male,34,3.0,152303.80,1,1,0,12128.69,0
8779,8780,15650322,Grigoryeva,701,France,Female,34,3.0,105588.66,1,0,1,74694.41,0
6545,6546,15572735,Chang,433,Spain,Male,27,2.0,0.00,2,1,1,153698.65,0
5950,5951,15806808,Hope,834,Germany,Female,57,8.0,112281.60,3,1,0,140225.14,1
3479,3480,15743714,Ch'ien,468,France,Male,46,7.0,91443.75,1,1,0,10958.18,0
1269,1270,15723339,Chin,554,France,Female,38,4.0,137654.05,2,1,1,172629.67,0
7083,7084,15636767,Yang,665,Spain,Female,32,10.0,0.00,1,1,1,22487.45,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [225]:
print(churn_data.duplicated().sum())

0


In [226]:
churn_data.columns = churn_data.columns.str.lower()
churn_data = churn_data.rename(columns={
    'rownumber': 'row_number', 
    'customerid': 'customer_id',
    'creditscore': 'credit_score',
    'numofproducts': 'num_of_products',
    'hascrcard': 'has_cr_card',
    'isactivemember': 'is_active_member',
    'estimatedsalary': 'estimated_salary',
})
print(churn_data.columns)

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')


In [227]:
print(churn_data['customer_id'].duplicated().sum())
churn_data['surname_lowercase'] = churn_data['surname'].str.lower()

print(churn_data['surname_lowercase'].duplicated().sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age']).sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age', 'geography']).sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age', 'geography', 'credit_score']).sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age', 'geography', 'credit_score', 'estimated_salary']).sum())

churn_data.drop(['surname_lowercase'], axis=1, inplace=True)







0
7069
596
241
2
0


In [228]:
print(churn_data['tenure'].isna().value_counts())
print(churn_data.query("tenure.isna() & is_active_member == 0").count())
churn_data['tenure'] = churn_data['tenure'].where((churn_data['tenure'].notna() & churn_data['is_active_member'] == 1) , 0.0) 
print()

print(churn_data.query("tenure.isna() & is_active_member == 0").count())
print(churn_data['tenure'].isna().value_counts())
churn_data['tenure'] = pd.to_numeric(churn_data['tenure'], errors='coerce')



False    9091
True      909
Name: tenure, dtype: int64
row_number          445
customer_id         445
surname             445
credit_score        445
geography           445
gender              445
age                 445
tenure                0
balance             445
num_of_products     445
has_cr_card         445
is_active_member    445
estimated_salary    445
exited              445
dtype: int64

row_number          0
customer_id         0
surname             0
credit_score        0
geography           0
gender              0
age                 0
tenure              0
balance             0
num_of_products     0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64
False    10000
Name: tenure, dtype: int64


In [229]:
#churn_data['exited'] = churn_data['exited'].astype('boolean')
print(churn_data.query("surname == 'Tan'").head(1))



     row_number  customer_id surname  credit_score geography gender  age  \
211         212     15624592     Tan           603    France   Male   31   

     tenure  balance  num_of_products  has_cr_card  is_active_member  \
211     8.0      0.0                2            1                 1   

     estimated_salary  exited  
211         169915.02       0  


In [230]:
display(churn_data.sample(10))
churn_data.info()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
9961,9962,15566543,Aldridge,573,Spain,Male,44,0.0,0.00,2,1,0,107124.17,0
8271,8272,15815645,Akhtar,481,France,Male,37,8.0,152303.66,2,1,1,175082.20,0
5980,5981,15781710,Carey,558,Spain,Female,31,0.0,0.00,2,1,0,166720.28,0
9934,9935,15774586,West,692,Germany,Female,43,10.0,118588.83,1,1,1,161241.65,1
4829,4830,15692599,Chiemela,687,France,Male,34,0.0,128270.56,1,1,0,191092.62,0
1129,1130,15685309,Souter,669,France,Female,35,7.0,0.00,1,1,1,49108.23,1
591,592,15604044,Mitchell,700,France,Male,38,0.0,134811.30,1,1,0,1299.75,0
5597,5598,15815552,Ferguson,670,France,Female,42,6.0,112333.63,1,1,1,65706.86,0
6674,6675,15759034,Li Fonti,654,France,Male,36,0.0,112262.84,1,1,0,12873.39,0
6710,6711,15701037,Barton,578,France,Male,39,0.0,0.00,2,1,0,70563.90,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            10000 non-null  float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [231]:
#churn_data = pd.get_dummies(churn_data['geography'], reset_index=True)
churn_data_ohe = pd.get_dummies(churn_data, drop_first=True)


churn_train1, churn_test = train_test_split (churn_data_ohe, test_size=0.20, train_size = 0.80, random_state=12345, shuffle=True)
churn_train, churn_valid = train_test_split (churn_train1, test_size=0.25, train_size = 0.75, random_state=12345, shuffle=True)

display(churn_train.head(10))
churn_train.info()
display(churn_test.head(10))
churn_test.info()
display(churn_valid.head(10))
churn_valid.info()

#print(pd.get_dummies(data['Gender']).head())


features_train = churn_train.drop(['exited'], axis=1)
target_train = churn_train['exited']
features_valid = churn_valid.drop(['exited'], axis=1)
target_valid = churn_valid['exited']
features_test = churn_test.drop(['exited'], axis=1)
target_test = churn_test['exited']

,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,...,surname_Zotova,surname_Zox,surname_Zubarev,surname_Zubareva,surname_Zuev,surname_Zuyev,surname_Zuyeva,geography_Germany,geography_Spain,gender_Male
492,493,15624170,639,38,4.0,81550.94,2,0,1,118974.77,...,0,0,0,0,0,0,0,0,0,0
6655,6656,15703763,554,44,0.0,85304.27,1,1,1,58076.52,...,0,0,0,0,0,0,0,0,0,1
4287,4288,15657529,714,53,1.0,99141.86,1,1,1,72496.05,...,0,0,0,0,0,0,0,1,0,1
42,43,15687946,556,61,2.0,117419.35,1,1,1,94153.83,...,0,0,0,0,0,0,0,0,0,0
8178,8179,15809654,707,46,7.0,127476.73,2,1,1,146011.55,...,0,0,0,0,0,0,0,0,0,0
2127,2128,15703482,710,34,0.0,134260.36,2,1,0,147074.67,...,0,0,0,0,0,0,0,1,0,1
2051,2052,15792222,712,37,0.0,106881.50,2,0,0,169386.81,...,0,0,0,0,0,0,0,0,0,0
6712,6713,15724838,599,43,0.0,0.00,1,1,0,170347.10,...,0,0,0,0,0,0,0,0,0,0
4994,4995,15585580,796,52,0.0,167194.36,1,1,1,62808.93,...,0,0,0,0,0,0,0,1,0,0
9523,9524,15790322,660,32,0.0,114668.89,1,1,0,84605.00,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 492 to 3295
Columns: 2945 entries, row_number to gender_Male
dtypes: float64(3), int64(8), uint8(2934)
memory usage: 17.3 MB


,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,...,surname_Zotova,surname_Zox,surname_Zubarev,surname_Zubareva,surname_Zuev,surname_Zuyev,surname_Zuyeva,geography_Germany,geography_Spain,gender_Male
7867,7868,15697201,640,46,3.0,0.00,1,1,1,156260.08,...,0,0,0,0,0,0,0,0,1,0
1402,1403,15613282,757,29,0.0,130306.49,1,1,0,77469.38,...,0,0,0,0,0,0,0,0,0,1
8606,8607,15694581,807,42,5.0,0.00,2,1,1,74900.90,...,0,0,0,0,0,0,0,0,1,1
8885,8886,15815125,668,45,4.0,102486.21,2,1,1,158379.25,...,0,0,0,0,0,0,0,0,1,1
6494,6495,15752846,699,28,7.0,0.00,2,1,1,22684.78,...,0,0,0,0,0,0,0,0,0,1
9628,9629,15596624,662,22,9.0,0.00,2,1,1,44377.65,...,0,0,0,0,0,0,0,0,0,0
7459,7460,15583724,645,29,4.0,0.00,2,1,1,74346.11,...,0,0,0,0,0,0,0,0,1,0
3964,3965,15714981,476,37,4.0,0.00,1,1,1,55775.84,...,0,0,0,0,0,0,0,0,0,1
7000,7001,15739548,775,28,0.0,111167.70,1,1,0,149331.01,...,0,0,0,0,0,0,0,0,0,1
4470,4471,15593795,516,53,0.0,156674.20,1,1,0,118502.34,...,0,0,0,0,0,0,0,1,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 7867 to 694
Columns: 2945 entries, row_number to gender_Male
dtypes: float64(3), int64(8), uint8(2934)
memory usage: 5.8 MB


,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,...,surname_Zotova,surname_Zox,surname_Zubarev,surname_Zubareva,surname_Zuev,surname_Zuyev,surname_Zuyeva,geography_Germany,geography_Spain,gender_Male
2358,2359,15665438,669,43,1.0,163159.85,1,0,1,15602.80,...,0,0,0,0,0,0,0,0,0,1
8463,8464,15661162,526,49,0.0,0.00,1,1,0,114539.67,...,0,0,0,0,0,0,0,0,1,1
163,164,15680772,721,36,2.0,0.00,2,1,1,106977.80,...,0,0,0,0,0,0,0,0,1,0
3074,3075,15609653,614,44,0.0,118715.86,1,1,0,133591.11,...,0,0,0,0,0,0,0,1,0,0
5989,5990,15601811,668,53,0.0,110240.04,1,0,0,183980.56,...,0,0,0,0,0,0,0,0,0,0
6230,6231,15616954,592,71,4.0,0.00,2,0,1,17013.54,...,0,0,0,0,0,0,0,0,0,1
6973,6974,15583394,659,39,8.0,106259.63,2,1,1,198103.32,...,0,0,0,0,0,1,0,1,0,1
5313,5314,15794939,783,72,5.0,121215.90,2,1,1,105206.48,...,0,0,0,0,0,0,0,0,0,0
1131,1132,15733974,500,37,0.0,125822.21,1,1,0,111698.00,...,0,0,0,0,0,0,0,0,1,1
3369,3370,15643294,703,33,8.0,190566.65,1,1,1,79997.14,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 2358 to 4995
Columns: 2945 entries, row_number to gender_Male
dtypes: float64(3), int64(8), uint8(2934)
memory usage: 5.8 MB


In [236]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accurate_score = accuracy_score(target_valid, predictions_valid)
    print('max_depth =', depth, ': ', end='')
    print(accurate_score)

max_depth = 1 : 0.8045
max_depth = 2 : 0.8325
max_depth = 3 : 0.843
max_depth = 4 : 0.8485
max_depth = 5 : 0.8555
max_depth = 6 : 0.86
max_depth = 7 : 0.8565
max_depth = 8 : 0.853
max_depth = 9 : 0.8445


In [237]:
decision_model = DecisionTreeClassifier(random_state=12345, max_depth=6)
decision_model.fit(features_train, target_train)
predictions_valid = decision_model.predict(features_valid)
accurate_score = accuracy_score(target_valid, predictions_valid)
print(accurate_score)

0.86


In [240]:
best_score = 0
best_est = 0

for est in range(40, 57):
    random_model = RandomForestClassifier(random_state=12345, n_estimators=est)
    random_model.fit(features_train, target_train)
    score = random_model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print("Accuracy of the best model on the validation set (n_estimators = {}): {}".format(best_est, best_score))

Accuracy of the best model on the validation set (n_estimators = 56): 0.8625


In [235]:
logistic_model =  LogisticRegression(random_state=12345, solver='liblinear')
logistic_model.fit(features_train, target_train)
score_train = logistic_model.score(
    features_train, target_train 
)  
score_valid = logistic_model.score(
  features_valid ,target_valid
)  

print(
    "Accuracy of the logistic regression model on the training set:",
    score_train,
)
print(
    "Accuracy of the logistic regression model on the validation set:",
    score_valid,
)

Accuracy of the logistic regression model on the training set: 0.7968333333333333
Accuracy of the logistic regression model on the validation set: 0.8045


In [242]:
test_model = RandomForestClassifier(random_state=12345, n_estimators=56) #selected based on accuracy determined above
test_model_fit = test_model.fit(features_train, target_train)

predictions_test = test_model.predict(features_test)
target_test = target_test.reset_index(drop=True)

correct = 0
for i in range(len(target_test)):
    if target_test[i] == predictions_test[i]:
        correct += 1
accuracy = correct / len(target_test)

print('Accuracy:', accuracy)

Accuracy: 0.838


In [246]:
confusion = confusion_matrix(target_test, predictions_test)
imbalanced_f1 = f1_score(target_test, predictions_test)
print(imbalanced_f1)
print(confusion)

0.4563758389261745
[[1540   33]
 [ 291  136]]
